In [ ]:
import keras.backend as K
from keras.layers import Layer, RNN
from keras.utils import tf_utils
import tensorflow_datasets as tfds
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Dense
from tensorflow.keras.initializers import GlorotUniform, Orthogonal, Zeros
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import numpy as np
import os

In [ ]:
VOCAB_SIZE = 2000
OOV = "<OOV>"
MAX_SEQUENCE_LENGTH = 1000
BATCH = 32
LEARNING_RATE = 0.001
EPOCHS = 10

In [ ]:
imdb = tfds.load('imdb_reviews', as_supervised=True)
train, test = imdb['train'], imdb['test']
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVSAWBT/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVSAWBT/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
training_sentences=[]
training_labels=[]
testing_sentences=[]
testing_labels=[]
for sentence,label in train:
    training_sentences.append(str(sentence.numpy()))
    training_labels.append(label.numpy())
for sentence,label in test:
    testing_sentences.append(str(sentence.numpy()))
    testing_labels.append(label.numpy())
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [ ]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences=tokenizer.texts_to_sequences(training_sentences)
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
training_padded=pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
testing_padded=pad_sequences(testing_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_dataset = tf.data.Dataset.from_tensor_slices((training_padded, training_labels_final))
test_dataset = tf.data.Dataset.from_tensor_slices((testing_padded, testing_labels_final))
train_dataset = train_dataset.batch(BATCH, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH, drop_remainder=True)

In [ ]:
class ELSTMCell(Layer):
  def __init__(self, units, epochs, **kwargs):
    self.state_size = [units, units]
    self.units = units
    self.h_sp = tf.Variable(
        [[.0] * units for _ in range(units)], trainable=False)
    self.exp_sum = tf.Variable(
        [[.0] * units for _ in range(units)], trainable=False)
    self.e_t = tf.Variable(
        [[.0] * units for _ in range(units)], dtype=tf.float64, trainable=False)
    self.ro_min = tf.Variable(.0001, dtype=tf.float64, trainable=False)
    self.ro_max = tf.Variable(.01, dtype=tf.float64, trainable=False)
    self.cur_epoch = tf.Variable(1, dtype=tf.float64, trainable=False)
    self.epochs = epochs
    super(ELSTMCell, self).__init__(**kwargs)

  def get_config(self):
    config = super().get_config()
    config.update({
        "units": self.units,
        "epochs": self.epochs
    })
    return config

  def set_current_epoch(self, val):
    self.cur_epoch.assign(val)

  def clear_accumulators(self):
    self.h_sp.assign([[.0] * self.units for _ in range(self.units)])
    self.exp_sum.assign([[.0] * self.units for _ in range(self.units)])
    self.e_t.assign([[.0] * self.units for _ in range(self.units)])

  @tf_utils.shape_type_conversion
  def build(self, input_shape):
    input_dim = input_shape[-1]

    self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                  name='kernel',
                                  initializer=GlorotUniform())

    self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                            name='recurrent_kernel',
                                            initializer=Orthogonal())

    self.bias = self.add_weight(shape=(self.units * 4,),
                                name='bias',
                                initializer=Zeros())

    self.built = True

  @tf.function
  def call(self, inputs, states, training=None):
    h_tm1 = states[0]
    c_tm1 = states[1]

    self.exp_sum.assign_add(tf.math.exp(tf.transpose(h_tm1) * self.h_sp))
    inputs_i = inputs
    inputs_f = inputs
    inputs_c = inputs
    inputs_o = inputs

    k_i, k_f, k_c, k_o = tf.split(
        self.kernel, num_or_size_splits=4, axis=1)

    x_i = K.dot(inputs_i, k_i)
    x_f = K.dot(inputs_f, k_f)
    x_c = K.dot(inputs_c, k_c)
    x_o = K.dot(inputs_o, k_o)

    b_i, b_f, b_c, b_o = tf.split(self.bias, num_or_size_splits=4, axis=0)

    x_i = K.bias_add(x_i, b_i)
    x_f = K.bias_add(x_f, b_f)
    x_c = K.bias_add(x_c, b_c)
    x_o = K.bias_add(x_o, b_o)

    h_tm1_i = h_tm1
    h_tm1_f = h_tm1
    h_tm1_c = h_tm1
    h_tm1_o = h_tm1

    x = (x_i, x_f, x_c, x_o)
    h_tm1 = (h_tm1_i, h_tm1_f, h_tm1_c, h_tm1_o)

    c, o = self._compute_carry_and_output(x, h_tm1, c_tm1)
    self.h_sp.assign_add(states[0])
    h = o * tf.keras.activations.tanh(c)

    return h, [h, c]

  @tf.function
  def _compute_carry_and_output(self, x, h_tm1, c_tm1):
    x_i, x_f, x_c, x_o = x
    h_tm1_i, h_tm1_f, h_tm1_c, h_tm1_o = h_tm1

    i = tf.keras.activations.hard_sigmoid(
      x_i + K.dot(h_tm1_i, self.recurrent_kernel[:, :self.units]))

    es = self.emotion_estimator(h_tm1_i)
    self.emotion_modulator(x_o, es)
    f = tf.keras.activations.hard_sigmoid(x_f + K.dot(
      h_tm1_f, self.recurrent_kernel[:, self.units:self.units * 2]))
    f = tf.cast(self.e_t, dtype=tf.float32) * f
    o = tf.keras.activations.hard_sigmoid(
      x_o + K.dot(h_tm1_o, self.recurrent_kernel[:, self.units * 3:]))
    a = tf.keras.activations.tanh(x_c + K.dot(
      h_tm1_c, self.recurrent_kernel[:, self.units * 2:self.units * 3]))

    c = f * c_tm1 + i * a
    return c, o

  @tf.function
  def emotion_estimator(self, h_t):
    exp_new = tf.math.exp(tf.transpose(h_t) * self.h_sp)
    p = exp_new / self.exp_sum
    return tf.math.reduce_mean(p) > .3

  @tf.function
  def emotion_modulator(self, x_o, es):
    to_f64 = lambda x: tf.cast(x, dtype=tf.float64)
    ro_dif = self.ro_max - self.ro_min
    ep = (self.epochs - self.cur_epoch) / self.epochs
    ro = to_f64(ro_dif) * ep + self.ro_min
    ro_h = ro * to_f64(tf.math.abs(x_o - self.h_sp))
    if es: self.e_t.assign(self.e_t + ro_h)
    else: self.e_t.assign(self.e_t - ro_h)
    self.normalize_e_t()

  @tf.function
  def normalize_e_t(self):
    min_e = tf.reduce_min(self.e_t)
    max_e = tf.reduce_max(self.e_t)
    if max_e > min_e:
      normalized = 2. * (self.e_t - min_e) / (max_e - min_e) - 1.
      self.e_t.assign(normalized)

In [ ]:
class EpochSetterCallback(tf.keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs=None):
    for l in self.model.layers:
      if isinstance(l, RNN):
        l.cell.clear_accumulators()
        l.cell.set_current_epoch(epoch)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = tf.keras.models.load_model('/content/gdrive/MyDrive/model2', custom_objects={'ELSTMCell': ELSTMCell})

In [ ]:
def lr_time_based_decay(epoch, lr):
  return lr * 1 / (1 + LEARNING_RATE / EPOCHS * epoch)

history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=(test_dataset),
    callbacks=[
      tf.keras.callbacks.LearningRateScheduler(lr_time_based_decay),
      EpochSetterCallback()
    ]
)

Epoch 1/5
781/781 [==============================] - 870s 1s/step - loss: 0.2736 - accuracy: 0.8910 - val_loss: 0.3394 - val_accuracy: 0.8614 - lr: 9.9452e-04
Epoch 2/5
781/781 [==============================] - 861s 1s/step - loss: 0.2438 - accuracy: 0.9044 - val_loss: 0.4720 - val_accuracy: 0.8087 - lr: 9.9442e-04
Epoch 3/5
781/781 [==============================] - 867s 1s/step - loss: 0.2466 - accuracy: 0.9034 - val_loss: 0.4902 - val_accuracy: 0.7989 - lr: 9.9422e-04
Epoch 4/5
781/781 [==============================] - 877s 1s/step - loss: 0.2132 - accuracy: 0.9180 - val_loss: 0.3548 - val_accuracy: 0.8714 - lr: 9.9392e-04
Epoch 5/5
781/781 [==============================] - 878s 1s/step - loss: 0.2059 - accuracy: 0.9219 - val_loss: 0.3717 - val_accuracy: 0.8698 - lr: 9.9352e-04


In [ ]:
model.save('/content/gdrive/MyDrive/model1', save_format='tf')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/model1/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/model1/assets
